In [18]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
import numpy as np    # for mathematical operations
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
import os

import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from moviepy.editor import *
from collections import deque
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.datasets.mnist import load_data
from numpy import reshape
import matplotlib.pyplot as plt
import os
import glob
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing import image
from keras.models import *
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout, UpSampling2D
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model


# dataset_directory = '/Users/parthkalathia/Desktop/NEW_DATA'
dataset_directory = '/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2'
# classes_list = ['TALKING', 'HEADDOWN', 'YAWNING', 'MOBILE', 'NORMAL']
classes_list =  ['NORMAL']

In [2]:
def preprocess_video(video_path, input_shape):
    # Load video and get properties
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Initialize array to store frames
    all_frames = np.zeros((frame_count, input_shape[1], input_shape[2], input_shape[3]))
    frames = np.zeros((input_shape[0], input_shape[1], input_shape[2], input_shape[3]))
        
    # Read frames and resize
    for i in range(frame_count):
        ret, frame = cap.read()
        frame_resized = cv2.resize(frame, (input_shape[2], input_shape[1]))
        all_frames[i] = frame_resized
    
    # extract equal distance frames from all_frames
    j = 0
    distance_frame = frame_count//input_shape[0]
    for i in range(input_shape[0]):
        frames[i] = all_frames[j]
        j = j + distance_frame

    
    # Normalize pixel values to [0, 1]
    frames = frames / 255.0
    
    # Close video capture
    cap.release()
    
    return frames

In [3]:
# from matplotlib import pyplot as plt
# plt.imshow(frames[10], interpolation='nearest')
# plt.show()

NameError: name 'frames' is not defined

In [10]:
def create_dataset(input_shape):
    import random
    # Declaring Empty Lists to store the features and labels values.
    features = []
    labels = []
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(classes_list):
        print(f'Extracting Data of Class: {class_name}')
        
        # Getting the list of video files present in the specific class name directory
        files_list = os.listdir(os.path.join(dataset_directory, class_name))
        print(files_list)

        # Iterating through all the files present in the files list
        for file_name in files_list:
            
            if file_name == '.DS_Store':
                continue
            # Construct the complete video path
            video_file_path = os.path.join(dataset_directory, class_name, file_name)
            print(video_file_path)

            # Calling the frame_extraction method for every video file path
            frames = preprocess_video(video_file_path, input_shape)
          

            # Appending the frames to a temporary list.
            features.append(frames)
        
            labels.append(class_index)

    # Converting the features and labels lists to numpy arrays
    features = np.array(features)
    labels = np.array(labels) 
    
    return features, labels  


In [ ]:
features, labels = create_dataset((16,256,256,3))

Extracting Data of Class: NORMAL
['N5.mp4', 'N4.mp4', 'N6.mp4', 'N7.mp4', 'N3.mp4', '.DS_Store', 'N2.mp4', 'N1.mp4', 'N23.mp4', 'N22.mp4', 'N20.mp4', 'N21.mp4', 'N19.mp4', 'N18.mp4', 'N24.mp4', 'N17.mp4', 'N15.mp4', 'N10.mp4', 'N11.mp4', 'N13.MOV', 'N12.MOV', 'N9.mp4', 'N8.mp4']
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N5.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N4.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N6.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N7.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N3.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N2.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N1.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N23.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N22.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N20.mp4
/Users/

In [8]:
print("Length of features : ", len(features))
print("Length of labels : ", len(labels))

Length of features :  2
Length of labels :  2


In [10]:
print("shape of features : ", features.shape)
print("shape of labels : ", labels.shape)

shape of features :  (50, 16, 256, 256, 3)
shape of labels :  (50,)


In [11]:
one_hot_encoded_labels = to_categorical(labels)

In [12]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(features, one_hot_encoded_labels, random_state=23)

In [16]:
def create_model():
  
    model = Sequential()
    sample_shape=(16,256,256,3)
    model.add(Conv3D(64, kernel_size=(3, 3, 3) ,activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape ,padding='same'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(BatchNormalization())
    
    model.add(GlobalAveragePooling3d())
    model.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.4))
    model.add(Dense(5, activation='softmax'))
    model.summary()
    return model

model = create_model()
print("Model Created Successfully!")

# Input shape

# 5+D tensor with shape: batch_shape + (channels, conv_dim1, conv_dim2, conv_dim3) 
# if data_format='channels_first' or 5+D tensor with 
# shape: batch_shape + (conv_dim1, conv_dim2, conv_dim3, channels) if data_format='channels_last'.

# Output shape

# 5+D tensor with shape: batch_shape + (filters, new_conv_dim1, new_conv_dim2, new_conv_dim3) 
# if data_format='channels_first' or 5+D tensor with 
# shape: batch_shape + (new_conv_dim1, new_conv_dim2, new_conv_dim3, filters) if data_format='channels_last'. new_conv_dim1, new_conv_dim2 and new_conv_dim3 values might have changed due to padding.



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_5 (Conv3D)           (None, 16, 256, 256, 32)  2624      
                                                                 
 conv3d_6 (Conv3D)           (None, 16, 256, 256, 64)  55360     
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 8, 128, 128, 64)  0         
 3D)                                                             
                                                                 
 conv3d_7 (Conv3D)           (None, 8, 128, 128, 128)  221312    
                                                                 
 conv3d_8 (Conv3D)           (None, 8, 128, 128, 256)  884992    
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 4, 64, 64, 256)   0         
 3D)                                                  

In [17]:
#Adding Early Stopping Callback 9824855926-sahdev bhai
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

# Adding loss, optimizer and metrics values to the model.
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [ ]:
model_training_history = model.fit(X_train, Y_train , shuffle = True, epochs = 5 , batch_size= 10, validation_split=0.1, callbacks = [early_stopping_callback])

Epoch 1/5
